# Gentle introduction to Finite Difference

# What is PDE? 



Consider a function $u$ that depends on two variables, $x$ and $y$. A partial differential equation (PDE) involving $u$ can be written as:

\begin{equation}
F\left(x, y, u, \frac{\partial u}{\partial x}, \frac{\partial u}{\partial y}, \frac{\partial^2 u}{\partial x^2}, \frac{\partial^2 u}{\partial x \partial y}, \frac{\partial^2 u}{\partial y^2}, \ldots\right) = 0
\end{equation}

where:
- $F$ is a given function.
- $u = u(x, y)$ is the unknown function to be solved for.
- $\frac{\partial u}{\partial x}$ and $\frac{\partial u}{\partial y}$ are first-order partial derivatives of $u$.
- $\frac{\partial^2 u}{\partial x^2}$, $\frac{\partial^2 u}{\partial x \partial y}$, and $\frac{\partial^2 u}{\partial y^2}$ are second-order partial derivatives of $u$, and so on for higher-order derivatives.


Solving a PDE typically involves finding a function $u(x, y)$ that satisfies the equation in a region of interest, subject to some boundary conditionent}


The finite difference method is a way to solve problems that involve changes, like how something moves or changes shape over time, by breaking them down into small, manageable pieces. Imagine you're trying to understand how a car accelerates. Instead of trying to capture every tiny detail at every moment, you break down the car's journey into small, equally spaced time intervals. For each interval, you look at how the car's speed changes.

Here's how it works in simple terms:

1. **Discretization**: You divide the time (or space) into small steps or intervals. Think of this like measuring the car's speed every second instead of trying to know its speed at every possible instant.

2. **Approximation**: For each interval, you estimate how much the quantity you're interested in (like velocity) changes. This is akin to noticing that the car goes a little faster each second, rather than needing a perfect formula for its speed.

3. **Solution**: By adding up all these small changes, you can figure out the overall behavior over time. If you know how much faster the car gets every second, you can estimate its speed after any number of seconds.

In more technical terms, the finite difference method approximates derivatives, which are mathematical tools that measure how a quantity changes. Instead of calculating the derivative exactly (which can be complex or impossible in many real-world scenarios), the method uses the values of the function at specific points to estimate these changes. This technique is very useful in solving differential equations, which are equations that describe how things change, and are commonly found in physics, engineering, and other fields.

By breaking down complex, continuous problems into simpler, discrete pieces, the finite difference method allows us to solve a wide range of problems that would otherwise be too difficult to tackle.



### Finite Difference Method

1. **General Approach**: The finite difference method (FDM) is a numerical technique used to solve differential equations, including partial differential equations (PDEs) that arise in various scientific and engineering disciplines, including finance. It approximates derivatives by using finite differences and iteratively solves for the value of a function over a domain.

2. **Application in Finance**: In finance, FDM can be applied to solve the Black-Scholes PDE or other more complex PDEs for various types of options and financial derivatives, not limited to European options. It's particularly useful for American options (which can be exercised any time before expiration) and exotic options that have features making analytical solutions difficult or impossible.

3. **Flexibility**: The method is flexible and can handle a wide range of boundary conditions and complex financial products. It can adapt to various shapes of the underlying payoff and accommodate features like early exercise (American options).

4. **Computational Aspect**: FDM involves discretizing the time and asset price space and iteratively solving the resulting system of equations. This process can be computationally intensive, especially for multi-dimensional problems (e.g., options on multiple assets).

### Black-Scholes Model

1. **Specific Model**: The Black-Scholes model is a mathematical model for pricing an options contract. Specifically, it gives an analytical expression for the price of European call and put options (i.e., options that can only be exercised at expiration) under certain assumptions, such as log-normal distribution of stock prices, constant volatility, and risk-free interest rate.

2. **Foundation**: The Black-Scholes formula is derived from the Black-Scholes Partial Differential Equation (PDE), which describes how the option's price evolves over time. Unlike FDM, which numerically solves PDEs, the Black-Scholes model provides a closed-form solution for European options.

3. **Assumptions**: The model is based on several simplifying assumptions, including no dividends paid out by the underlying asset, constant volatility and interest rates, and the ability to continuously hedge option positions. These assumptions are not always realistic, limiting the model's applicability to real-world scenarios.

4. **Use and Limitations**: While elegant and widely used, the Black-Scholes model's assumptions (especially regarding constant volatility) can make it less accurate for predicting real-world option prices. The model does not directly apply to American options or more complex derivative structures without modifications or extensions.

### Comparison Summary

- **Purpose and Scope**: FDM is a general numerical method for solving differential equations, while the Black-Scholes model specifically provides an analytical solution for European option prices under certain assumptions.
- **Flexibility vs. Specificity**: FDM offers flexibility to handle a broader range of option types and conditions, whereas the Black-Scholes model offers a simple, closed-form solution for a specific set of option types and conditions.
- **Computational Approach**: FDM requires setting up and solving a discrete approximation of the problem, which can be computationally intensive. The Black-Scholes model, when applicable, provides a direct formula requiring input of parameters.

In practice, the choice between using FDM or the Black-Scholes model depends on the specific financial instrument being modeled, the required accuracy, and the computational resources available.

In [24]:
import numpy as np
import pandas as pd

class FiniteDifference:
    def __init__(self, df):
        """
        Initializes the OptionPricer with a DataFrame containing option and underlying asset data.
        
        :param df: DataFrame with columns for underlying asset price ('UNDERLYING_LAST'), 
                   strike price ('STRIKE'), risk-free rate ('r'), days to expiration ('DTE'), 
                   and volatility ('sigma').
        """
        self.df = df
   
        

    def finite_difference_option_price(self, S, X, r, T, sigma, N, M, option_type=None, american=True):
        """Prices an option using the finite difference method.
        
        :param S: Current price of the underlying asset.
        :param X: Exercise (strike) price of the option.
        :param r: Risk-free interest rate.
        :param T: Time to expiration of the option (in years).
        :param sigma: Volatility of the underlying asset's price.
        :param N: Number of time steps in the finite difference grid.
        :param M: Number of asset price steps in the finite difference grid.
        :param option_type: Type of the option ('call' or 'put').
        :param american: Boolean indicating if the option is American (True) or European (False).
        :return: The priced value of the option.
        """
        dt = T/N
        dx = sigma * np.sqrt(dt)
        pu = 0.5 * dt * ((sigma**2 / dx**2) + (r - 0.5 * sigma**2) / dx)
        pm = 1 - dt * (sigma**2 / dx**2 + r)
        pd = 0.5 * dt * ((sigma**2 / dx**2) - (r - 0.5 * sigma**2) / dx)
        grid = np.zeros((M+1, N+1))
        stock_prices = S * np.exp(np.arange(-M/2, M/2 + 1) * dx)
        
        # Terminal conditions
        if option_type == 'call':
            grid[:, -1] = np.maximum(stock_prices - X, 0)
        elif option_type == 'put':
            grid[:, -1] = np.maximum(X - stock_prices, 0)

        # Iterate backwards in time
        for i in range(N-1, -1, -1):
            for j in range(1, M):
                grid[j, i] = pu * grid[j+1, i+1] + pm * grid[j, i+1] + pd * grid[j-1, i+1]
                if american:
                    # Check for early exercise
                    if option_type == 'call':
                        grid[j, i] = max(grid[j, i], stock_prices[j] - X)
                    elif option_type == 'put':
                        grid[j, i] = max(grid[j, i], X - stock_prices[j])

        # Return the option price at S
        return grid[M//2, 0]

    def price_options(self, N=50, M=50, american=True, option_type='call'):
        """
        Applies the finite difference option pricing method to each row in the DataFrame.
        
        :param N: Number of time steps.
        :param M: Number of price steps.
        :param american: Boolean indicating if options are American.
        :param option_type: Specifies the option type to price ('call' or 'put').
        """
        # Apply the pricing method to each row in the DataFrame
        self.df['Option_Price_FD'] = self.df.apply(
            lambda row: self.finite_difference_option_price(
                S=row['UNDERLYING_LAST'],
                X=row['STRIKE'],
                r=row['r'],
                T=row['DTE'] / 365,  # Convert DTE to years
                sigma=row['sigma'],
                N=N,
                M=M,
                option_type=option_type,  # Assumes 'call' or 'put' is specified in the DataFrame
                american=american  # Whether to consider it as an American option for early exercise
            ),
            axis=1
        )

# Generate a synthetic dataset
np.random.seed(42)  # For reproducibility
data = {
    'UNDERLYING_LAST': np.random.uniform(90, 110, 10),  # Random stock prices between 90 and 110
    'STRIKE': np.random.uniform(95, 105, 10),  # Random strike prices between 95 and 105
    'r': np.random.uniform(0.01, 0.05, 10),  # Random risk-free rates between 1% and 5%
    'DTE': np.random.randint(30, 365, 10),  # Random days to expiration between 30 and 365
    'sigma': np.random.uniform(0.15, 0.3, 10),  # Random volatilities between 15% and 30%
    'option_type': ['call'] * 5 + ['put'] * 5  # 5 calls and 5 puts
}
df = pd.DataFrame(data)

pricer = FiniteDifference(df)

# Price the options
pricer.price_options(N=50, M=50, american=True, option_type='call')  # For calls
pricer.price_options(N=50, M=50, american=True, option_type='put')  # For puts

df[['UNDERLYING_LAST', 'STRIKE', 'r', 'DTE', 'sigma', 'option_type', 'Option_Price_FD']]


,UNDERLYING_LAST,STRIKE,r,DTE,sigma,option_type,Option_Price_FD
0,97.490802,95.205845,0.034474,336,0.207812,call,5.460152
1,109.014286,104.699099,0.015580,164,0.152395,call,2.317877
2,104.639879,103.324426,0.021686,50,0.184634,call,2.120192
3,101.973170,97.123391,0.024654,358,0.186154,call,4.324857
4,93.120373,96.818250,0.028243,196,0.252490,call,8.314451
5,93.119890,96.834045,0.041407,303,0.241499,put,8.869193
6,91.161672,98.042422,0.017987,118,0.274979,put,9.664673
7,107.323523,100.247564,0.030569,345,0.176005,put,3.259988
8,102.022300,99.319450,0.033697,43,0.208659,put,1.607170
9,104.161452,97.912291,0.011858,271,0.177335,put,3.273878
